### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Summer 2022 ###

# Road Trip Optimizer #

#### Alex Gilmore (asgilmore@wisc.edu), Nandan Venkatesan (nvenkatesan2@wisc.edu), and Brendan Zimmer (btzimmer@wisc.edu)

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-Model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-Discussion)
1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

add from google doc

## 2. Mathematical Model ##

We are utilizing an MCNF framework to solve our primary problem. The source and sink will be identical since we are interested in only round trips and we will be implementing a minimum trip length to prevent the model from cutting the trip short to reduce cost.

let p be a list of national parks and our starting location

let $x_{ij}$ be a matrix representing if an arc is chosen $\forall i \in p, \forall j \in p$

let $e_i$ be a vector of entrance fees $\forall i \in p$

let $l_i$ be a vector of est. lodging fees $\forall i \in p$

let $s_i$ be a vector of length of stays $\forall i \in p$

let $c_i$ be a vector of the total lodging and entrance fee cost $\forall i \in p$

let $g_{ij}$ be a matrix of gas prices $\forall i \in p, \forall j \in p$

let $D_{ij}$ be a matrix of  arc distances (miles) $\forall i \in p, \forall j \in p$

let $T_{ij}$ be a matrix of arc times (days) $\forall i \in p, \forall j \in p$

let $b_k$ be a vector of "supply/demand" $\forall k \in p$

let M be the maximum days allowed to travel

let m be the minimum days allowed to travel


\begin{align*}
\underset{x}{\max} \ & - \underset{(ij) \in p}\Sigma\ c_{i} x_{ij} - \underset{(ij) \in p}\Sigma\ g_{ij} x_{ij}  & \\
\text{s.t.} \ & \underset{(ij) \in p}\Sigma\ s_{i} x_{ij} + \underset{(ij) \in p}\Sigma\ T_{ij} x_{ij} \le M &\\
& - \underset{(ij) \in p}\Sigma\ s_{i} x_{ij} - \underset{(ij) \in p}\Sigma\ T_{ij} x_{ij} \le - m &\\
& \underset{j \in p}\Sigma\ x_{kj} - \underset{i \in p}\Sigma\ x_{ik} \le b_k,\ \forall\ k \in p &\\
& - \underset{j \in p}\Sigma\ x_{kj} + \underset{i \in p}\Sigma\ x_{ik} \le - b_k,\ \forall\ k \in p &\\
\end{align*}



In [ ]:
#constraints
# travel max
# travel min
# park selection

Variables:

- $x_{ij}$ = $\left\{ \begin{array}{ll}
        1 \ \mbox{if arc $x_{ij}$ is in road trip path} \\
        0 \ \mbox{otherwise} & 
    \end{array} \right\}$

Constraints:
1. The total time traveled must be less than the maximum travel time.


2. The total time traveled must be more than the minimum desired travel time. This prevents the model from decreasing the trip length in an effort to reduce the trip cost. 


3. This is a flow balance constraint to direct our road trip path. Only one arc (road) may enter a node (park) and only one arc may leave. This constraint is split into two halves (the third and fourth constraint listed) to display the mathematical model in standard form.

Objective Function:
- This is minimzing the collective cost of visiting parks + the gas cost of traveling between all parks. 

## 3. Solution ##

### data entry

In [197]:
using Clp, Gurobi, JuMP, NamedArrays, CSV, DataFrames

In [346]:
df = DataFrame(CSV.File("park_data.csv"))

# push!(df, ("UW_Madison", "", 0, 0, 0, 3.5))

push!(df, ("source", "", 0, 0, 0, 3.5))
# push!(df, ("sink", "", 0, 0, 0, 3.5))

pTemp = df[!, "park name"]
p = Array{Symbol}(undef, length(pTemp))

# format parks list to symbols
for i in [1:1:length(pTemp);]
    pTemp[i] = replace(pTemp[i], " " => "_", "–" => "_", "." => "")
    p[i] = Symbol(pTemp[i])
end 

e = NamedArray(df[!, "entrance fee"], (p))

l = NamedArray(df[!, "lodging"], (p))

s = NamedArray(df[!, "est. length of stay"], (p))

parkGas = NamedArray(df[!, "gas price"], (p)) 

;

$c = s * (l + e)$

In [347]:
c = s .* (l + e)
;

In [348]:
dist = DataFrame(CSV.File("arc_data_dist_all.csv"))

# distO = DataFrame(CSV.File("origin_data_dist.csv"))

time = DataFrame(CSV.File("arc_data_time_all.csv"))

# timeO = DataFrame(CSV.File("origin_data_time.csv"))

# dist.UW_Madison = distO[!, 2]
# time.UW_Madison = timeO[!, 2]

size(time[1:49, 2:50])
size(p)

(49,)

In [351]:
D = Float64.(NamedArray(Matrix(dist[1:49,2:50]), (p, p), ("entering", "leaving")))
T = NamedArray(Matrix(time[1:49,2:50]), (p, p), ("entering", "leaving"))
;

In [352]:
T = T ./ 86400 # convert time from seconds to days
;

$g_{ij} =  \frac{parkGas_i + parkGas_j}{2} * \frac{1}{milesPerGallon} * D_{ij}$

In [353]:
# calculate the gas cost matrix

milesPerGallon = 20.0

g = copy(D)

for i in p
    for j in p
        g[i, j] = (parkGas[i] + parkGas[j]) / (2milesPerGallon) * g[i, j]
    end
end

println("acadia to yosemite distance: ", D[:Acadia, :Yosemite], " miles")
println("acadia to yosemite estimated gas cost: \$", g[:Acadia, :Yosemite])
println("acadia to yosemite time : ", T[:Acadia, :Yosemite], " days")

acadia to yosemite distance: 3100.0 miles
acadia to yosemite estimated gas cost: $795.15
acadia to yosemite time : 1.9795138888888888 days


In [355]:
# T[:source, :Gateway_Arch] + T[:Gateway_Arch, :Hot_Springs] + T[:Hot_Springs, :Indiana_Dunes] + T[:Indiana_Dunes, :sink]

In [356]:
# set travel length boundaries

maxTravel = 14
minTravel = 7
;

In [382]:
# supply / demand

# b = zeros(length(p))
# b[49] = -1 # set sink
# b[50] = 1 # set source

# b = NamedArray(b, (p))

# set n (total number of parks)
n = length(p)
# n = 5
;

### model 

In [387]:
m = Model(Gurobi.Optimizer)

@variable(m, x[p, p], Bin)
@variable(m, 1 <= u[p] <= n)

@objective(m, Min, sum(c[i] * x[i, j] for i in p for j in p) 
    + sum(g[i, j] * x[i, j] for i in p for j in p))

;

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-08


In [388]:
# travel time constraints
@constraint(m, sum(s[i] * x[i, j] for i in p for j in p) 
    + sum(T[i, j] * x[i, j] for i in p for j in p) <= maxTravel)

@constraint(m, sum(s[i] * x[i, j] for i in p for j in p) 
    + sum(T[i, j] * x[i, j] for i in p for j in p) >= minTravel)

# balance constraint
# @constraint(m, balance[k in p], sum(x[k, j] for j in p) - sum(x[i, k] for i in p) == b[k])

# no self loops constraint
@constraint(m, x_constr[i in p], x[i, i] == 0)

# no source --> sink loops constraints
# @constraint(m, sink_constr, x[:sink, :source] == 0)
# @constraint(m, source_constr, x[:source, :sink] == 0)

# start at source, end at sink constraints
@constraint(m, start_constr, sum(x[:source, j] for j in p) == 1)
@constraint(m, end_constr, sum(x[i, :source] for i in p) == 1)

# MTZ logical constraint
@constraint(m, MTZ[i in p, j in p[2:end]], u[i] - u[j] + n*x[i, j] <= n - 1)

# one out edge, one in edge constraints (at most)
@constraint(m, x_row_constr[k in p], sum(x[k, j] for j in p) <= 1)
@constraint(m, x_col_constr[k in p], sum(x[i, k] for i in p) <= 1)

# if sum(s[i] * x[i, j] for i in p for j in p) 
#     + sum(T[i, j] * x[i, j] for i in p for j in p) >= minTravel
# @constraint(m, return_yet[i in p, j in p], sum(s[i] * x[i, j] for i in p for j in p) 
#                         + sum(T[i, j] * x[i, j] for i in p for j in p) + x[i,j] >= 7)
# then x[i, :sink] == 1
# @constraint(m, min[i in p], )

# if x[i, j] == 1, then x[j, i] == 0
# @constraint(m, one_way[i in p, j in p], x[i, j] <= x[j, i])



1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [:Acadia, :Arches, :Badlands, :Big_Bend, :Biscayne, :Black_Canyon_of_the_Gunnison, :Bryce_Canyon, :Canyonlands, :Capitol_Reef, :Carlsbad_Caverns  …  :Sequoia, :Shenandoah, :Theodore_Roosevelt, :Voyageurs, :White_Sands, :Wind_Cave, :Yellowstone, :Yosemite, :Zion, :source]
And data, a 49-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x_col_constr[Acadia] : x[Acadia,Acadia] + x[Arches,Acadia] + x[Badlands,Acadia] + x[Big_Bend,Acadia] + x[Biscayne,Acadia] + x[Black_Canyon_of_the_Gunnison,Acadia] + x[Bryce_Canyon,Acadia] + x[Canyonlands,Acadia] + x[Capitol_Reef,Acadia] + x[Carlsbad_Caverns,Acadia] + x[Congaree,Acadia] + x[Crater_Lake,Acadia] + x[Cuyaho

In [389]:
optimize!(m)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2503 rows, 2450 columns and 16709 nonzeros
Model fingerprint: 0xcfb48ca7
Variable types: 49 continuous, 2401 integer (2401 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [4e+01, 4e+03]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 1980.3605000
Presolve removed 245 rows and 150 columns
Presolve time: 0.07s
Presolved: 2258 rows, 2300 columns, 15448 nonzeros
Variable types: 46 continuous, 2254 integer (2254 binary)
Found heuristic solution: objective 1528.5750000

Root relaxation: objective 5.416834e+02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  541.68343    0    1 1528.575

In [390]:
counter = 0

println("cost of road trip: \$", objective_value(m), "\n")

for i in p
    for j in p
        if value(x[i, j]) > 0
            counter += 1
            println("arc ", counter, ": ", i, " --> ", j)
        end
    end
end
 println("\n")
for i in p
    for j in p
        if value(x[i, j]) > 0
            counter += 1
            println(x[i,j])
        end
    end
end

cost of road trip: $614.652

arc 1: Biscayne --> Everglades
arc 2: Gateway_Arch --> source
arc 3: Great_Smoky_Mountains --> Congaree
arc 4: source --> Indiana_Dunes


x[Biscayne,Everglades]
x[Gateway_Arch,source]
x[Great_Smoky_Mountains,Congaree]
x[source,Indiana_Dunes]


In [377]:
for i in p
    println(value.(u[i]), " ", i)
end

1.0 Acadia
21.0 Arches
45.0 Badlands
11.000000000000002 Big_Bend
5.000000000000002 Biscayne
18.0 Black_Canyon_of_the_Gunnison
23.0 Bryce_Canyon
20.0 Canyonlands
22.0 Capitol_Reef
13.000000000000002 Carlsbad_Caverns
4.000000000000002 Congaree
37.0 Crater_Lake
50.0 Cuyahoga_Valley
30.0 Death_Valley
6.000000000000002 Everglades
9.000000000000002 Gateway_Arch
41.0 Glacier
26.0 Grand_Canyon
43.0 Grand_Teton
24.0 Great_Basin
15.000000000000002 Great_Sand_Dunes
7.000000000000002 Great_Smoky_Mountains
12.000000000000002 Guadalupe_Mountains
10.000000000000002 Hot_Springs
49.0 Indiana_Dunes
29.0 Joshua_Tree
32.0 Kings_Canyon
35.0 Lassen_Volcanic
8.000000000000002 Mammoth_Cave
19.0 Mesa_Verde
38.0 Mount_Rainier
3.0000000000000013 New_River_Gorge
40.0 North_Cascades
39.0 Olympic
27.0 Petrified_Forest
34.0 Pinnacles
36.0 Redwood
17.0 Rocky_Mountain
28.0 Saguaro
33.0 Sequoia
2.0000000000000013 Shenandoah
46.0 Theodore_Roosevelt
47.0 Voyageurs
14.000000000000002 White_Sands
44.0 Wind_Cave
42.0 Yellow